In [1]:
# libreries for pre-processing 
import numpy                      as np
import matplotlib.pyplot          as plt
import pandas                     as pd
from   sklearn.model_selection    import KFold
from   sklearn.impute             import SimpleImputer
from sklearn.preprocessing        import LabelEncoder


# libreries for pre-Machine Learning 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler, MinMaxScaler
from sklearn.decomposition   import PCA

# libreries for Machine Learning 
from sklearn.svm                import SVC
from sklearn.neighbors          import KNeighborsClassifier
from sklearn.tree               import DecisionTreeClassifier
from sklearn.ensemble           import RandomForestClassifier
from sklearn.metrics            import confusion_matrix, accuracy_score, f1_score,matthews_corrcoef,precision_score
from sklearn.neural_network     import MLPClassifier
from sklearn.model_selection    import KFold,GridSearchCV



In [2]:
dataset = pd.read_csv('breast_cancer.csv')

In [3]:
dataset

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0,2
1,1002945,5.0,4.0,4.0,5.0,7.0,10.0,3.0,2.0,1.0,2
2,1015425,3.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0,2
3,1016277,6.0,8.0,8.0,1.0,3.0,4.0,3.0,7.0,1.0,2
4,1017023,4.0,1.0,1.0,3.0,2.0,1.0,3.0,1.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...
678,776715,3.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0,2
679,841769,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2
680,888820,5.0,10.0,10.0,3.0,7.0,3.0,8.0,10.0,2.0,4
681,897471,4.0,8.0,6.0,4.0,3.0,4.0,10.0,6.0,1.0,4


In [4]:
Features = dataset.iloc[:,1:-1]
Labels   = dataset.iloc[:,-1]
print(Features.shape)
print(Labels.shape)

(683, 9)
(683,)


In [5]:
if Features.isna().values.any():
    print("There are NaN values")

There are NaN values


Check NaN values

In [6]:
imputer  = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(Features) # for only the columns with numerical values
Features = imputer.transform(Features)
Features = pd.DataFrame(Features)

In [7]:
if Features.isna().values.any():
    print("There are NaN values")

In [8]:
Features

,0,1,2,3,4,5,6,7,8
0,5.0,1.0,1.0,1.0,2.0,1.0,3.0,1.0,1.0
1,5.0,4.0,4.0,5.0,7.0,10.0,3.0,2.0,1.0
2,3.0,1.0,1.0,1.0,2.0,2.0,3.0,1.0,1.0
3,6.0,8.0,8.0,1.0,3.0,4.0,3.0,7.0,1.0
4,4.0,1.0,1.0,3.0,2.0,1.0,3.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...
678,3.0,1.0,1.0,1.0,3.0,2.0,1.0,1.0,1.0
679,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0
680,5.0,10.0,10.0,3.0,7.0,3.0,8.0,10.0,2.0
681,4.0,8.0,6.0,4.0,3.0,4.0,10.0,6.0,1.0


In [9]:
le = LabelEncoder()
Labels = le.fit_transform(Labels)

In [10]:
# split dataset  --> 80% training and 20% test
X_train, X_test, y_train, y_test = train_test_split(Features, Labels, test_size = 0.2, random_state = 42)

In [11]:
classifiers = []

In [12]:
# Dataset Std based on mean and sd of the train set 
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test  = sc.transform(X_test)

In [13]:
# Principal Component Analisys 
pca = PCA(n_components=0.98) #6     # number of components to have 98% variance of data --> based on train test
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test) 

In [14]:
K_outer = 5
outer_cv = KFold(n_splits=K_outer,shuffle=True,random_state=42)

In [15]:
classifiers = [SVC(kernel = 'sigmoid', random_state = 42),
               KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2),
               DecisionTreeClassifier(),
               RandomForestClassifier(n_estimators=100)
               ]

clf_names = ['SVC','kNN','decisionTree','random_forest']

TRAINING WITHOUT OPTIMIZATION

In [22]:

metric_accuracies = np.zeros([K_outer,len(classifiers)])
metric_prediction = np.zeros([K_outer,len(classifiers)])
metric_f1score    = np.zeros([K_outer,len(classifiers)])
metric_mcc        = np.zeros([K_outer,len(classifiers)])


for number_classifier in range(len(classifiers)):
    
    print('*******************')
    cl_name = clf_names[number_classifier]
    print(f'Classificatore: {cl_name}')

    if cl_name == 'kNN':
        # Parameters to be optimized for the K-NEAREST NEIGHBOUR classifier
        n_neighbors = list(range(1, 20))
        param_grid = {'n_neighbors': n_neighbors, 'algorithm':['ball_tree', 'kd_tree', 'brute']}
    if cl_name == 'SVC':
        # Parameters to be optimized for the support vector machine classifier
        C_range = np.logspace(-3, 1, 50)
        gamma_range = np.logspace(-4,1,50)
        param_grid = {'C': C_range, 'gamma':gamma_range}
    if cl_name == 'decisionTree':
        # Parameters to be optimized for the decision tree classifier
        param_grid = {'criterion':['gini', 'entropy', 'log_loss']}
    if cl_name == 'random_forest':
        # Parameters to be optimized for the random forest classifier
        n_estimators_range = range(30,500,10)
        param_grid = {'n_estimators': n_estimators_range, 'criterion':['gini', 'entropy', 'log_loss']}
    
    clf_counter = 0
    
      # Train/Test loop - OUTER LOOP
    for train_index, valid_index in outer_cv.split(X_train, y_train):

        print("")
        print(f'Outer cycle equal to. {clf_counter}')

        # Outer Fold Split
        X_train_out, X_valid = X_train[train_index], X_train[valid_index]
        y_train_out, y_valid = y_train[train_index], y_train[valid_index]

        classifier = classifiers[number_classifier]
        grid = GridSearchCV(estimator = classifier,param_grid = param_grid,cv = 5,scoring = 'matthews_corrcoef')

        grid.fit(X_train_out,y_train_out)

        y_pred = grid.predict(X_valid)

        cm                             = confusion_matrix(y_valid, y_pred)
        print(cm)
        metric_accuracies[clf_counter,number_classifier] = accuracy_score(y_valid, y_pred)
        metric_prediction[clf_counter,number_classifier] = precision_score(y_valid, y_pred)
        metric_f1score   [clf_counter,number_classifier] = f1_score(y_valid, y_pred)
        metric_mcc       [clf_counter,number_classifier] = matthews_corrcoef(y_valid, y_pred)

        clf_counter += 1

#best_on_outer = np.argmax(metric_accuracies) 

np.argmax(metric_mcc,dim=1)

*******************
Classificatore: SVC

Outer cycle equal to. 0
[[68  2]
 [ 3 37]]

Outer cycle equal to. 1


KeyboardInterrupt: 

TRAINING WITH OPTIMIZATION